In [5]:
%run _bootstrap.py
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pyarrow, pandas, sys; \
           print('pyarrow', pyarrow.__version__, \
                 '\npandas ', pandas.__version__, \
                 '\npython ', sys.version.split()[0])

pyarrow 16.1.0 
pandas  2.3.0 
python  3.11.8


In [6]:
# Option A – simplest: don't reload any compiled libs
%autoreload 2
%aimport -numpy -pandas -pyarrow -ccxt   # the leading '-' = *exclude*



In [7]:
from src.data_loader import get_asset_df
df = get_asset_df("BTC/USDT")
print(df.tail())

                                open       high        low      close  \
timestamp                                                               
2025-07-18 15:30:00+00:00  117760.00  118021.71  117480.00  117860.25   
2025-07-18 16:00:00+00:00  117860.24  118054.92  117530.68  117607.57   
2025-07-18 16:30:00+00:00  117607.57  118012.73  117479.92  117553.42   
2025-07-18 17:00:00+00:00  117553.41  117996.49  117377.00  117924.01   
2025-07-18 17:30:00+00:00  117924.00  117973.52  117885.04  117950.91   

                              volume  
timestamp                             
2025-07-18 15:30:00+00:00  402.47649  
2025-07-18 16:00:00+00:00  411.96789  
2025-07-18 16:30:00+00:00  331.94160  
2025-07-18 17:00:00+00:00  278.33861  
2025-07-18 17:30:00+00:00   15.33341  


In [8]:
df.index.freq  # should show <30 * Minutes>

In [11]:
# 1-hour bars for your anomaly detector
import numpy as np
df_1h = df.resample("1h").agg(
    {"open": "first",
     "high": "max",
     "low": "min",
     "close": "last",
     "volume": "sum"}
)

# Add log-returns, z-scores, etc.
df_1h["log_ret"] = np.log(df_1h["close"]).diff()

print(df_1h)

                                open       high        low      close  \
timestamp                                                               
2025-06-27 22:00:00+00:00  107052.01  107190.21  107052.01  107100.70   
2025-06-27 23:00:00+00:00  107100.70  107171.23  106961.70  107047.59   
2025-06-28 00:00:00+00:00  107047.58  107176.68  106847.59  107064.19   
2025-06-28 01:00:00+00:00  107064.20  107129.33  106811.51  107066.19   
2025-06-28 02:00:00+00:00  107066.19  107200.00  107015.70  107110.00   
...                              ...        ...        ...        ...   
2025-07-18 13:00:00+00:00  119442.43  119442.44  118500.00  118773.38   
2025-07-18 14:00:00+00:00  118773.38  119242.63  117315.69  118139.99   
2025-07-18 15:00:00+00:00  118139.99  118265.96  117480.00  117860.25   
2025-07-18 16:00:00+00:00  117860.24  118054.92  117479.92  117553.42   
2025-07-18 17:00:00+00:00  117553.41  117996.49  117377.00  117950.91   

                               volume   log_ret  
